In [1]:
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def generate_date_string(start_date, end_date):
    date_format = "%Y/%m/%d"
    dates_strings = []

    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)

    current_date = start
    while current_date <= end:
        dates_strings.append(current_date.strftime(date_format))
        current_date += timedelta(days=1)

    return dates_strings

In [3]:
def generate_date_array(start_date, end_date):
    date_format = "%Y/%m/%d"
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)

    # Create an empty list to store the generated dates
    date_list = []

    # Generate the dates between the start and end dates
    current_date = start
    while current_date <= end:
        formatted_date = current_date.strftime(date_format).replace('/', '')
        date_list.append(formatted_date)
        current_date += timedelta(days=1)

    # Convert the list to a NumPy array
    date_array = np.array(date_list)

    return date_array

In [4]:
def get_day_of_week(date_string):
    try:
        date = datetime.strptime(date_string, "%Y/%m/%d")
        day_of_week = date.strftime("%A")
        return day_of_week
    except ValueError:
        return "Invalid date format. Please provide a date in the format YYYY/MM/DD."

In [5]:
years = np.arange(2014,2023,1)
start_dates = []
end_dates = []
for year in years:
    start_dates.append(f"{year}/01/01")
    end_dates.append(f"{year}/12/31")

In [6]:
d = {year: pd.DataFrame() for year in years}
for year in years:
    start_date = start_dates[year-2014]
    end_date = end_dates[year-2014]
    dates_strings = generate_date_string(start_date, end_date)
    dates_array = generate_date_array(start_date, end_date)
    dates_df = pd.DataFrame(dates_strings)
    dates_df.columns = ['dates']
    dates_df['weeks'] = dates_df['dates'].apply(get_day_of_week)
    dates_list_df = pd.DataFrame(dates_array)
    d[year] = pd.concat([dates_list_df,dates_df],axis=1)

In [21]:
d_weekday = {year: pd.DataFrame() for year in years}
for year in years:
    not_sun = d[year]["weeks"] != "Sunday"
    not_sun_weekday = d[year][not_sun]
    not_sat = not_sun_weekday["weeks"] != "Saturday"
    d_weekday[year] = not_sun_weekday[not_sat]
    d_weekday[year].columns = ['dates', 'weeks', 'dates1']

In [22]:
year_of = int(input("Enter chosen year"))

In [23]:
dates = d_weekday[year_of]['dates'].values
pages = []
for date in dates:
    expression = f"https://www.bondspot.pl/tbsp_index_archiwum?date={date}"
    pages.append(expression)

In [28]:
datyes = []
opening_values = []
closing_values = []
cells = []
timeout_seconds = 300

In [29]:
for page in pages:
    response = requests.get(page, timeout = timeout_seconds)
    soup = BeautifulSoup(response.content, "html.parser")
    rows = soup.find_all('tr')
    for row in rows:
        try:
            cells = row.find_all('td')
        except AttributeError:
            continue
    datyes.append(cells[0].text.strip())
    opening_values.append(cells[1].text.strip())
    closing_values.append(cells[2].text.strip())

In [43]:
d_final = {year: pd.DataFrame() for year in years}
df = pd.DataFrame(datyes)
df_2 = pd.DataFrame(map(float, opening_values))
df_3 = pd.DataFrame(map(float, closing_values))
d_final[year_of] = pd.concat([df, df_2, df_3], axis=1)
d_final[year_of].columns = ['dates', 'opening_values', 'closing_values']
d_final[year_of] = d_final[year_of][d_final[year_of].closing_values > 28]

In [44]:
d_final[year_of].to_csv(f"data/{year_of}.csv")